# UrbanSoundsClassification: AIYard

Önişleme.ipynb

AIYard ekibi

### Kütüphanelerin içe aktarımı

In [1]:
import cv2 as cv
import os
import pandas as pd
import numpy as np

### Spektrogramların okunması
Google Colaboratory'de spektrogram verilerini okuyup önişlemeye tabi tutarken hafıza limitini aştığımız için sıkıntı yaşadık.

Bunun önüne geçmek için spektrogramları verilen Drive linkinden indirdik ve lokal cihazlarımızda önişlemeye tabi tuttuk.

Alttaki kodda yapılacak tek değişiklik walk() metoduna verilecek olan klasörün yoludur.

In [2]:
X = [] # Okunan spektrogram matrislerini tuttuğumuz liste
Y = [] # Okunan spektrogramların sınıflarını tuttuğumuz liste
img_size = 64 # Spektrogramların küçültülecekleri boyut

for path, subdirs, files in os.walk("/home/ferit/repos/UrbanSoundsClassification/spectrograms/"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = cv.imread(file, 0) # Dosyayı grayscale olacak şekilde açmak için 0 pozisyonel argümanını veriyoruz
        spectrogram = cv.resize(spectrogram, (img_size, img_size), interpolation=cv.INTER_LINEAR) # Görüntüyü bilineer interpolasyonla küçültüyoruz 
        spectrogram = spectrogram.astype("float64") # Normalizasyonda yüksek doğruluklu ondalıklı sayılar elde etmek için matris veri tipini float64 olarak değiştiriyoruz 
        spectrogram /= 255 # Piksel değerleri maksimum 255 olacağı için tüm hücreleri 255'e bölerek değerleri 0 ile 1 arasına çekiyoruz
        X.append(spectrogram) # X listesine spektrogramı ekliyoruz
        Y.append(int(file.split('/')[6])) # Y listesine spektrogramın sınıfını ekliyoruz

In [3]:
print(len(X))
print(len(Y))

8735
8735


In [4]:
X_train = X[0:6900]

In [5]:
Y_train = Y[0:6900]

In [6]:
train = pd.DataFrame()
train["X_train"] = X_train
train["Y_train"] = Y_train
train.to_csv("train.csv")

In [7]:
X_val = X[6900:7800]
Y_val = Y[6900:7800]

In [8]:
validation = pd.DataFrame()
validation["X_val"] = X_val
validation["Y_val"] = Y_val
validation.to_csv("validation.csv")

In [9]:
X_test = X[7800:8735]
Y_test = Y[7800:8735]

In [10]:
test = pd.DataFrame()
test["X_test"] = X_test
test["Y_test"] = Y_test
test.to_csv("test.csv")